In [3]:
import pytorch_lightning as pl
from pl_bolts.models.self_supervised import SwAV
import wandb
from torchvision.transforms import v2 as transforms
from torch.utils.data import Dataset
import pandas as pd
import skimage as ski
import os
import data
from data import MedakaDataset
from torch.utils.data import random_split, DataLoader
import yaml
from pl_bolts.transforms.self_supervised.swav_transforms import (
    SwAVTrainDataTransform,
    SwAVEvalDataTransform
)

In [4]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ey267 (ey267-university-of-cambridge). Use `wandb login --relogin` to force relogin


True

In [5]:
from pytorch_lightning.loggers import WandbLogger

In [6]:
os.environ['WANDB_DATA_DIR'] = '/hps/nobackup/birney/users/esther/wandb/artifacts/staging/'
os.environ['WANDB_ARTIFACT_DIR'] = '/hps/nobackup/birney/users/esther/wandb/artifacts/'
os.environ['WANDB_CACHE_DIR'] = '/hps/nobackup/birney/users/esther/wandb/.cache/'
os.environ['WANDB_TIMEOUT'] = '120'

In [7]:
def load_config(config_path):
    with open(config_path, "r") as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    return config

config = load_config("/nfs/research/birney/users/esther/medaka-img/src_files/wandb_yaml/vae-v0.yaml")

In [8]:
batch_size = 128
train_len = 800

# model
model = SwAV(
    gpus=1,
    num_samples=train_len,
    dataset='medakadataset_2024-10-03',
    batch_size=batch_size,
    num_crops=(2,4),
    learning_rate=0.0001,
    num_prototypes=50
)

In [9]:
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [10]:
# Need a DataModule class as input for SwAV?
class MedakaDataModule(pl.LightningDataModule):
    def __init__(self, train_transforms, val_transforms, batch_size):
        super().__init__()
        self.train_len = train_len
        self.batch_size = batch_size

        self.train_transforms = None if train_transforms is None else train_transforms
        self.val_transforms = None if val_transforms is None else val_transforms

    def train_dataloader(self):
        dataset = MedakaDataset(data_csv='/nfs/research/birney/users/esther/medaka-img/src_files/train_set_2024-10-03.csv',
                                        direction_csv='/nfs/research/birney/users/esther/medaka-img/scripts/left-facing-fish.csv',
                                        src_dir='/nfs/research/birney/users/esther/medaka-img/src_files/train_2024-10-03/',
                                        transform=self.train_transforms, 
                                        config=config)
        dataset_train, _ = random_split(dataset, [self.train_len, len(dataset) - self.train_len])
        return DataLoader(dataset_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        dataset = MedakaDataset(data_csv='/nfs/research/birney/users/esther/medaka-img/src_files/train_set_2024-10-03.csv',
                                        direction_csv='/nfs/research/birney/users/esther/medaka-img/scripts/left-facing-fish.csv',
                                        src_dir='/nfs/research/birney/users/esther/medaka-img/src_files/train_2024-10-03/',
                                        transform=self.val_transforms, 
                                        config=config)
        _, dataset_val = random_split(dataset, [self.train_len, len(dataset) - self.train_len])
        return DataLoader(dataset_val, batch_size=self.batch_size)

In [11]:
train_transforms = SwAVTrainDataTransform(
    size_crops=(224, 96),
    gaussian_blur=False
)

val_transforms = SwAVEvalDataTransform(
    size_crops=(224, 96),
    gaussian_blur=False
)

medaka_dm = MedakaDataModule(train_transforms, val_transforms, batch_size)

In [12]:
wandb_logger = WandbLogger(project="SWAV",
                            log_model='all')

# fit
trainer = pl.Trainer(precision=16, accelerator='auto', logger=wandb_logger)
trainer.fit(model, medaka_dm)

/hps/software/users/birney/esther/micromamba/envs/indigene-img/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /hps/software/users/birney/esther/micromamba/envs/in ...
  rank_zero_warn(
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/hps/software/users/birney/esther/micromamba/envs/indigene-img/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, 

Sanity Checking: 0it [00:00, ?it/s]

/hps/software/users/birney/esther/micromamba/envs/indigene-img/lib/python3.9/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/hps/software/users/birney/esther/micromamba/envs/indigene-img/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/hps/software/users/birney/esther/micromamba/envs/indigene-img/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:83: UserWarning: Trying to infer the `batch_size` fr

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]